# Recommendation Systems for Speedrun.com

# Recommendation by User ID using Bipartite Graph

This method of recommendation works by finding other users that have played the same game as a target user. The games that the other users have played are ranked by occurrence. Games that the target user has not played but the other users have played are then recommended in order.

In [3]:
import pandas as pd
import csv
import networkx as nx
import numpy as np

from collections import defaultdict, Counter
from operator import itemgetter

In [2]:
user_prefs_filename = "../data/users/user_preferences_with_metadata.csv"
user_prefs_df = pd.read_csv(user_prefs_filename)
user_prefs_df = user_prefs_df[(user_prefs_df['signup_date'].notna()) & (user_prefs_df['signup_date'] != "Null")]
user_prefs_df['signup_date'] = pd.to_datetime(user_prefs_df['signup_date'], format='%Y-%m-%dT%H:%M:%SZ')
user_prefs_df['signup_date'] = pd.to_datetime(user_prefs_df['signup_date'].dt.strftime('%Y-%m-%d'))
user_prefs_df = user_prefs_df[(user_prefs_df['signup_date'] < '2023-01-01')]

In [3]:
user_prefs_df.describe(include='all', datetime_is_numeric=True)

,user,signup_date,location,num_games,games
count,335322,335322,335322,335322.000000,335322
unique,335322,NaN,257,NaN,88806
top,j5wzz2qj,NaN,us,NaN,k6q4rqzd
freq,1,NaN,101439,NaN,5131
mean,NaN,2020-06-28 13:40:53.271780608,NaN,1.994465,NaN
min,NaN,2014-01-06 00:00:00,NaN,1.000000,NaN
25%,NaN,2019-09-16 00:00:00,NaN,1.000000,NaN
50%,NaN,2021-01-04 00:00:00,NaN,1.000000,NaN
75%,NaN,2021-09-25 00:00:00,NaN,2.000000,NaN
max,NaN,2022-12-31 00:00:00,NaN,2059.000000,NaN


In [4]:
print(f"Number of users: {user_prefs_df.count()['user']}")

Number of users: 335322


In [5]:
def find_games_from_df(df: pd.DataFrame):
    all_games = set()
    for users_games in list(df['games'].values):
        games = users_games.split(',')
        [all_games.add(game) for game in games]
    return all_games

In [6]:
print(f"Number of games: {len(find_games_from_df(user_prefs_df))}")

Number of games: 31425


In [7]:
exploded_games_df = user_prefs_df.copy()
exploded_games_df['games'] = exploded_games_df['games'].str.split(',')
exploded_games_df = exploded_games_df.explode('games').rename(columns = {'games': 'game_id', 'user':'user_id'})[['user_id', 'game_id']]

In [8]:
exploded_games_df.describe()

,user_id,game_id
count,668788,668788
unique,335322,31425
top,kj9521v8,k6q4rqzd
freq,2059,6979


In [9]:
bipartite_graph = nx.Graph()

# Users have a bipartite value of 0, games have a bipartite value of 1.
bipartite_graph.add_nodes_from(set(exploded_games_df['user_id'].values), bipartite=0)
bipartite_graph.add_nodes_from(set(exploded_games_df['game_id'].values), bipartite=1)
bipartite_graph.add_edges_from([(user, game) for user, game in zip(exploded_games_df['user_id'], exploded_games_df['game_id'])])

In [10]:
nx.is_bipartite(bipartite_graph)

True

In [11]:
del user_prefs_filename, user_prefs_df, exploded_games_df

### Overlapping Set Similarity

In [12]:
NUM_USERS = 335322
NUM_GAMES = 31425

In [13]:
def user_similarity(bipartite_graph: nx.Graph, user_a_id: str, user_b_id: str, total_item_nodes=NUM_GAMES) -> float:
    assert bipartite_graph.nodes[user_a_id]['bipartite'] == 0
    assert bipartite_graph.nodes[user_b_id]['bipartite'] == 0

    a_neighbours = bipartite_graph.neighbors(user_a_id)
    b_neighbours = bipartite_graph.neighbors(user_b_id)
    shared_nodes = set(a_neighbours).intersection(b_neighbours)

    return len(shared_nodes) / total_item_nodes

In [14]:
user_similarity(bipartite_graph, 'xyr917zj', 'j92k73o8')

6.364359586316626e-05

In [15]:
def most_similar_users(bipartite_graph: nx.Graph, user_id: str) -> tuple[list[str], float]:
    all_users = set([user for user, value in bipartite_graph.nodes(data=True) if value['bipartite'] == 0])
    assert len(all_users) == NUM_USERS, print(len(all_users))
    all_users.remove(user_id)
    similarities = defaultdict(float)
    for user in all_users:
        similarities[user] = user_similarity(bipartite_graph, user_id, user)

    max_similarity = max(similarities.values())
    return [user for user, similarity in similarities.items() if similarity == max_similarity], max_similarity

In [16]:
similar_users, similarity_value = most_similar_users(bipartite_graph, "xyr917zj")
similar_users[:5], similarity_value

(['kjpg5p2j', 'j965zvrj', 'jn3gv71x', '0jm34we8', '8ge6kzpj'],
 6.364359586316626e-05)

In [17]:
def recommend_games(bipartite_graph: nx.Graph, user_id: str) -> list[str]:
    similar_users, _ = most_similar_users(bipartite_graph, user_id)
    other_games = set()
    
    for user in similar_users:
        [other_games.add(game) for game in bipartite_graph.neighbors(user)]

    already_played_games = set(bipartite_graph.neighbors(user_id))
    return list(other_games.difference(already_played_games))

In [18]:
recommend_games(bipartite_graph, "xyr917zj")

['9d3rrxyd',
 'k6q4wjmd',
 'ldeyle13',
 '3dx20o41',
 'yd4w456e',
 'pd0wlq21',
 '46w305q1',
 'yd4ljx1e',
 'kdk9v26m',
 '268n576p',
 'w6j7vpx6',
 '46w0m91r',
 'y6547z0d',
 '268xe71p',
 'w6jlvz7d',
 'k6q4zozd',
 '9dow9e1p',
 '36983x0d',
 '9dojxo6p',
 '2689251p',
 'w6jlon7d',
 '3dx2mg41',
 'm1mxx5k6',
 'y65ey86e',
 'nd27e310',
 '268wol56',
 'kdkm89e1',
 'm1zj9px6',
 'k6qg8xdg',
 'kyd4gde4',
 'k6q4ov9d',
 'om1mpk12',
 'yd4z7k6e',
 'j1n8y0x1',
 'kdk44x1m',
 'v1pvo418',
 '9done31p',
 'j1ne7ll1',
 'o1y94826',
 'nd2e9erd',
 'k6q47lxd',
 '369p9g21',
 '4d79oel1',
 'pd0qrev1',
 'kdkyne1m',
 '3dx7np6y',
 'yd4og9p1',
 '4d70jn17',
 'k6qg2odg',
 'j1n48l6p',
 'k6qr091g',
 'yd4o4zp1',
 '9dolo7dp',
 '268wmk6p',
 'o6gnyy81',
 '3dx4l46y',
 '3dxwl41y',
 'y65ep06e',
 'm1mxgp62',
 '3dx0l5dy',
 'm1zjq506',
 '369pjmg1',
 '9d388kg1',
 'ldejwj13',
 'y6543l8d',
 'nd2elx3d',
 '9d3r5ygd',
 'yd4l2k1e',
 'yd4g2pde',
 'j1np0w6p',
 'v1pxg786',
 'pdv279r6',
 '9d3rnedl',
 '369e92dl',
 'w6jlw74d',
 '3698w3dl',
 '9d37o96l',

In [19]:
games_metadata_df = pd.read_csv('../data/games/metadata/all_games.csv')
recommended_games = recommend_games(bipartite_graph, "8qz5ovo8")
games_metadata_df[(games_metadata_df['game_id'].isin(recommended_games))].sort_values(by='num_runs').iloc[::-1]

,game_id,game_name,developers,release_date,created_date,num_categories,num_levels,num_runs,num_users,num_guests
5162,k6q4rqzd,Seterra,"4eppvoer,ne410dem",1997-01-01,2018-09-27T08:29:37Z,29,903,61962,7168,36
4924,o1y9wo6q,Super Mario 64,xv6dvx62,1996-06-23,None,6,15,33895,6118,452
23519,o1y9j9v6,Celeste,1zk4q26j,2018-01-25,2018-01-15T20:23:45Z,15,9,32317,5406,14
16295,yd478gde,Minecraft: Legacy Console Edition,"yzom2keq,yge00xep",2012-05-09,2015-01-29T23:42:25Z,11,12,19379,155,15646
15794,j1npme6p,Minecraft: Java Edition,k62d97ex,2011-11-18,2015-01-29T23:41:21Z,15,0,18329,6268,491
...,...,...,...,...,...,...,...,...,...,...
6927,9d3r03wd,Aconcagua,None,2000-06-01,2020-09-24T14:30:16Z,1,0,1,1,0
6466,76r3mev6,Disney Villains' Revenge,None,1999-09-18,2022-10-05T21:31:00Z,3,0,1,1,0
26477,m1mxgy46,Regina & Mac,None,2019-11-13,2020-02-15T16:20:11Z,3,0,1,1,0
4662,9d3rp4wd,The Shannara,ge0rv5ep,1995-12-01,2020-02-15T02:51:35Z,1,0,1,1,0


In [20]:
del NUM_GAMES, NUM_USERS, similar_users, similarity_value, games_metadata_df, recommended_games

### Overlapping Set Similarity with Limiting the Graph

There are two methods of limiting the number of user-item interactions in our bipartite graph. We can either use the mean and standard deviation of the `num_games` column, or limit based on the integer number of games played by a given user. For example, we can either use three standard deviations of the mean to have a cutoff value of `24.2 (3 s.f.)`, or we can use the value of `2` for users that have played only one game.

Using the method of standard deviations, we get a very similar output to the unlimited user-item interaction bipartite graph. We get popular games recommended most of the time. If we use the second approach, we get (anecdotally) more precise recommendations. However, the second method does not scale well, since we need to construct a different graph for each number of games played by each user. In reality, this isn't as bad as we think. Out of the 335,322 total users in our sample we can cover 306,371 users, or 91.4% (3 s.f.) of them with three graphs. 

In [26]:
def clean_user_preferences_df(df: pd.DataFrame) -> pd.DataFrame:
    df = df[(df['signup_date'].notna()) & (df['signup_date'] != "Null")]
    df['signup_date'] = pd.to_datetime(df['signup_date'], format='%Y-%m-%dT%H:%M:%SZ')
    df['signup_date'] = pd.to_datetime(df['signup_date'].dt.strftime('%Y-%m-%d'))
    df = df[(df['signup_date'] < '2023-01-01')]
    return df

def limit_number_games_user_preferences_df(df: pd.DataFrame, num_games: int) -> pd.DataFrame:
    return df[(df['num_games'] <= num_games)]

def explode_games_played(df: pd.DataFrame) -> pd.DataFrame:
    df['games'] = df['games'].str.split(',')
    return df.explode('games').rename(columns = {'games': 'game_id', 'user':'user_id'})

def recommendation_graph_for_n_games_played(df: pd.DataFrame, n: int) -> tuple[pd.DataFrame, nx.Graph]:
    df = clean_user_preferences_df(df)
    df = limit_number_games_user_preferences_df(df, n+1)
    df = explode_games_played(df)
    bipartite_graph = nx.Graph()
    bipartite_graph.add_nodes_from(set(df['user_id'].values), bipartite=0)
    bipartite_graph.add_nodes_from(set(df['game_id'].values), bipartite=1)
    bipartite_graph.add_edges_from([(user, game) for user, game in zip(df['user_id'], df['game_id'])])
    return df, bipartite_graph

In [27]:
user_prefs_df = pd.read_csv('../data/users/user_preferences_with_metadata.csv')
user_prefs_df, bipartite_graph = recommendation_graph_for_n_games_played(user_prefs_df, 5)

C:\Users\ajcm203\AppData\Local\Temp\ipykernel_8108\4036755379.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['signup_date'] = pd.to_datetime(df['signup_date'], format='%Y-%m-%dT%H:%M:%SZ')
C:\Users\ajcm203\AppData\Local\Temp\ipykernel_8108\4036755379.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['signup_date'] = pd.to_datetime(df['signup_date'].dt.strftime('%Y-%m-%d'))


In [28]:
def user_similarity(bipartite_graph: nx.Graph, total_item_nodes: int, user_a_id: str, user_b_id: str) -> float:
    assert bipartite_graph.nodes[user_a_id]['bipartite'] == 0
    assert bipartite_graph.nodes[user_b_id]['bipartite'] == 0

    a_neighbours = bipartite_graph.neighbors(user_a_id)
    b_neighbours = bipartite_graph.neighbors(user_b_id)
    shared_nodes = set(a_neighbours).intersection(b_neighbours)

    return len(shared_nodes) / total_item_nodes

def most_similar_users(bipartite_graph: nx.Graph, user_id: str) -> tuple[list[str], float]:
    all_users = set([user for user, value in bipartite_graph.nodes(data=True) if value['bipartite'] == 0])
    all_users.remove(user_id)

    total_item_nodes = 0
    for _, values in bipartite_graph.nodes(data=True):
        if values['bipartite'] == 1: total_item_nodes += 1

    similarities = defaultdict(float)
    for user in all_users:
        similarities[user] = user_similarity(bipartite_graph, total_item_nodes, user_id, user)

    max_similarity = max(similarities.values())
    return [user for user, similarity in similarities.items() if similarity == max_similarity], max_similarity

def recommend_games(bipartite_graph: nx.Graph, user_id: str) -> list[str]:
    similar_users, _ = most_similar_users(bipartite_graph, user_id)
    other_games = [game for user in similar_users for game in bipartite_graph.neighbors(user)]
    game_rankings = Counter(other_games)

    already_played_games = set(bipartite_graph.neighbors(user_id))

    try:
        [game_rankings.pop(game) for game in already_played_games]
    except KeyError:
        # If no other users in data set have played this game.
        pass

    ranked_games_in_order, _ = list(zip(*sorted(game_rankings.items(), key=itemgetter(1), reverse=True)))
    
    return ranked_games_in_order

In [29]:
games_metadata_df = pd.read_csv('../data/games/metadata/all_games.csv').rename(columns={'game_id': 'id'})

In [30]:
user = "x355n6qj"

played_games = list(bipartite_graph.neighbors(user))
print(f"Played games: {games_metadata_df[(games_metadata_df['id'].isin(played_games))].game_name.values}")

recommended_games = recommend_games(bipartite_graph, user)
print(f"Recommended games: {games_metadata_df.set_index('id').loc(axis=0)[recommended_games].game_name.values[:20]}")

Played games: ['Hello Neighbor' 'Super Mario Odyssey']
Recommended games: ["Baldi's Basics Category Extensions" 'Google Quick Draw'
 'Snipperclips: Cut it out  together!' 'Hello Neighbor 2' 'Clicker Heroes'
 'Minecraft: Java Edition' 'Island Saver'
 'Super Mario Odyssey Category Extensions' 'Cuphead'
 'Super Mario Sunshine' 'Marble Saga: Kororinpa'
 'The Legend of Zelda: The Wind Waker HD']


In [35]:
del games_metadata_df, played_games, recommended_games, user_prefs_df, bipartite_graph, user

# Recommendation Using a Game Similarity Matrix

This recommendation method works by creating a matrix of how users have rated different games. We construct this matrix by rating a game `1` if a user has played it, and `0` if not. We then normalise these values by making the sum of ratings by each user equal to `1`. This is also called [scaling to a unit length](https://en.wikipedia.org/wiki/Feature_scaling#Scaling_to_unit_length). We take the dot product of the matrix and the transposed matrix, and this gives us the similarity between each item in the data set. The method is taken from [here](https://towardsdatascience.com/recommender-systems-item-customer-collaborative-filtering-ff0c8f41ae8a). **This method does not scale very well**.

In [138]:
user_prefs_df = pd.read_csv('../data/users/user_preferences_with_metadata.csv')

In [139]:
def format_user_prefs_df_to_ratings(df: pd.DataFrame, number_users=-1) -> pd.DataFrame:
    tmp_df = df.copy()[:number_users]
    tmp_df['games'] = tmp_df['games'].str.split(',')
    tmp_df = tmp_df.explode('games').rename(columns = {'games': 'game_id', 'user':'user_id'})
    tmp_df['rating'] = 1
    return tmp_df[['user_id', 'game_id', 'rating']]

def construct_similarity_matrix(df: pd.DataFrame) -> pd.DataFrame:
    tmp_df = df.copy()
    tmp_df = pd.pivot_table(tmp_df, values='rating', index='user_id', columns='game_id')
    tmp_df = tmp_df.fillna(0.0)
    normalised_tmp_df = tmp_df / np.sqrt(np.square(tmp_df).sum(axis=0))
    return normalised_tmp_df.transpose().dot(normalised_tmp_df)

def construct_similar_games_df(df: pd.DataFrame, number_games: int) -> pd.DataFrame:
    similar_games_df = pd.DataFrame(index=df.columns, columns=range(0, number_games + 1))
    for i in range(0,len(df.columns)): 
        similar_games_df.iloc[i,:number_games+1] = df.iloc[0:,i].sort_values(ascending=False)[:number_games+1].index
    return similar_games_df.loc[:,1:]

def find_similar_games(df: pd.DataFrame, top_n_games=10):
    similarity_matrix = construct_similarity_matrix(df)
    similar_games_df = construct_similar_games_df(similarity_matrix, top_n_games)
    return similar_games_df

In [140]:
similar_games_df = pd.DataFrame()
generate = False

if generate:
    ratings_df = format_user_prefs_df_to_ratings(user_prefs_df, number_users=70000)
    similar_games_df = find_similar_games(ratings_df, 20)
    similar_games_df.to_csv('./test.csv')
else:
    similar_games_df = pd.read_csv('../data/users/similarity_recommendations_70000_users.csv').set_index('game_id')

In [141]:
len(similar_games_df.index)

28541

In [142]:
games_metadata_df = pd.read_csv('../data/games/metadata/all_games.csv').rename(columns={'game_id': 'id'})

In [144]:
game_id = "v1prkz68"
print(f"Played Game: {games_metadata_df.set_index('id').loc(axis=0)[game_id].game_name}")
recommended_games = similar_games_df.loc[game_id].values
print(f"Recommended games: {games_metadata_df.set_index('id').loc(axis=0)[recommended_games].game_name.values}")

Played Game: Resident Evil 7: Biohazard
Recommended games: ['RE7: Banned Footage DLC' 'RE7: Not a Hero'
 'Resident Evil 7: Biohazard Category Extensions' 'RE7: End of Zoe'
 'Resident Evil Village' 'Resident Evil 2 (2019)'
 'Resident Evil HD Remaster (Steam)' 'Resident Evil HD Remaster (Console)'
 'Resident Evil 3 (2020)' 'Silent Hill' 'Ghostwire: Tokyo' 'Silent Hill 2'
 'Resident Evil 6' 'Resident Evil: Survivor' 'Steelrising'
 'Resident Evil 3: Nemesis' 'The Evil Within 2' 'Silent Hills P.T.'
 "Hellblade: Senua's Sacrifice" 'SAW: The Game']


In [78]:
del game_id, generate, recommended_games, similar_games_df, user_prefs_df, games_metadata_df

# Recommendation using Node2Vec Embeddings

The idea behind using node2vec embeddings for recommendation is to predict future links that don't already exist. We can prove that this works for individual games recommendation by removing selected edges and using cosine similarity of embeddings to predict which edges should exist given this graph. We carry this on further by creating a pipeline to predict games to play when they are completely disconnected.

In [90]:
import pandas as pd
import networkx as nx
import csv

from sklearn.metrics.pairwise import cosine_similarity
from operator import itemgetter

In [114]:
def get_weighted_edges_from_csv(filename: str, filter=None) -> list[tuple[str, str, int]]:
    with open(filename, 'r', encoding='utf-8') as openfile:
        csv_reader = csv.reader(openfile)
        next(csv_reader)

        edges = list()
        for row in csv_reader:
            if filter is None:
                edges.append(tuple([row[0], row[1], int(row[2])]))
                continue
            
            if not filter.get(row[0]) or not filter.get(row[1]):
                continue

            edges.append(tuple([row[0], row[1], int(row[2])]))

    return edges

def predict_links(g: nx.DiGraph, embeddings_df: pd.DataFrame, target_node_id: str, number_links: int):
    target_node_embeddings = embeddings_df[embeddings_df.index == target_node_id]
    all_nodes = g.nodes()
    other_node_ids = [node_id for node_id in all_nodes if node_id not in list(graph.neighbors(target_node_id))]
    other_embeddings = embeddings_df[embeddings_df.index.isin(other_node_ids)]

    similarities = cosine_similarity(target_node_embeddings, other_embeddings)[0].tolist()
    nodes_and_similarity = dict(zip(other_embeddings.index.tolist(), similarities))
    nodes_and_similarity = sorted(nodes_and_similarity.items(), key=itemgetter(1), reverse=True)

    similar_nodes = [node[0] for node in nodes_and_similarity[1:number_links]]
    return similar_nodes

In [110]:
embeddings_df = pd.read_csv('../data/games/network/homophily.emb', delimiter=" ", skiprows=1, index_col=0, header=None).add_prefix("dim_")

In [124]:
graph = nx.DiGraph()
edgelist = get_weighted_edges_from_csv('../data/too_big/all_games_filtered.csv')
graph.add_weighted_edges_from(edgelist)

In [111]:
games_metadata_df = pd.read_csv('../data/games/metadata/all_games.csv').rename(columns={'game_id': 'id'})

In [147]:
edges_to_remove = [edge for edge in graph.edges() if edge[0] == "v1prkz68"][:20]
print(edges_to_remove)
edited_graph = graph.copy()
edited_graph.remove_edges_from(edges_to_remove)

[('v1prkz68', '9dowp2o1'), ('v1prkz68', 'm1zjg360'), ('v1prkz68', 'j1n5e91p'), ('v1prkz68', 'ldemon13'), ('v1prkz68', 'pdvmyodw'), ('v1prkz68', 'y65v00de'), ('v1prkz68', 'kdk83gdm'), ('v1prkz68', '3dx2w9y1'), ('v1prkz68', 'j1lewj6g'), ('v1prkz68', 'm1m99362'), ('v1prkz68', 'm1zjlkm6'), ('v1prkz68', 'yd4okqp1'), ('v1prkz68', '9d3xz0dl'), ('v1prkz68', 'k6qejz6g'), ('v1prkz68', 'm1zjx260'), ('v1prkz68', 'j1n29w1p'), ('v1prkz68', '268ge5dp'), ('v1prkz68', '369540dl'), ('v1prkz68', 'w6j0k2dj'), ('v1prkz68', '46w0x31r')]


In [148]:
predicted_links = predict_links(edited_graph, embeddings_df, "v1prkz68", 20)
print(f"Recommended games: {games_metadata_df.set_index('id').loc(axis=0)[predicted_links].game_name.values}")

Recommended games: ['RE Demake' 'Resident Evil HD Remaster (Console) Category Extension'
 'Heaven Dust' 'Wanted: Weapons Of Fate' 'Tokyo Dark' 'Penumbra: Requiem'
 'Lithium: Inmate 39' 'Blue Estate' 'Onigiri' 'BloodRayne: Betrayal'
 'Bright Memory - Episode 1' 'Project Sylpheed'
 'Devil May Cry 3: Special Edition - Category Extensions'
 "Lone Survivor: The Director's Cut" 'Resident Evil: Gaiden' 'RATUZ'
 'Crimson Snow' 'Vergils Downfall: Definitive Edition'
 'Awkward Dimensions Redux']
